In [181]:
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import math

In [182]:
movies = pd.read_csv("movie_metadata.csv")

In [183]:
movies

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660


### one hot encoding genre and plot keyword columns

In [184]:
genres = movies.genres

In [185]:
genres_lists = [x.split("|") for x in genres]

In [186]:
#genres_lists

In [187]:
movies["genres"] = genres_lists

In [188]:
#movies

In [189]:
movies["plot_keywords"]

0                  avatar|future|marine|native|paraplegic
1       goddess|marriage ceremony|marriage proposal|pi...
2                     bomb|espionage|sequel|spy|terrorist
3       deception|imprisonment|lawlessness|police offi...
4                                                     NaN
                              ...                        
5038               fraud|postal worker|prison|theft|trial
5039         cult|fbi|hideout|prison escape|serial killer
5040                                                  NaN
5041                                                  NaN
5042    actress name in title|crush|date|four word tit...
Name: plot_keywords, Length: 5043, dtype: object

In [190]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

In [191]:
movies_clean_genres = movies.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(movies.pop('genres')),
                index=movies.index,
                columns=mlb.classes_))

In [192]:
#movies.columns

In [193]:
#movies["plot_keywords"]

In [194]:
plot_keywords = list(movies["plot_keywords"])


In [195]:
plot_keywords = [str(keyword) for keyword in plot_keywords]
plot_keywords = [keyword.split("|") for keyword in plot_keywords]

In [196]:
#plot_keywords

In [197]:
movies["plot_keywords"] = plot_keywords

In [198]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

In [199]:
#movies

In [200]:
movies.rename(columns = {"color": "movie_color", "language": "movie_language"}, inplace = True)

In [201]:
og_columns = movies.columns

In [202]:
movies_clean_genres = movies.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(movies.pop('plot_keywords')),
                index=movies.index,
                columns=mlb.classes_))

In [203]:
#movies_clean_genres

### dropping nas

In [204]:
movies_no_na = movies_clean_genres.dropna()

In [205]:
#movies_no_na

In [206]:
string_columns = movies_no_na.select_dtypes(exclude=[np.number])
string_columns

,movie_color,director_name,actor_2_name,actor_1_name,movie_title,actor_3_name,movie_imdb_link,movie_language,country,content_rating
0,Color,James Cameron,Joel David Moore,CCH Pounder,Avatar,Wes Studi,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,English,USA,PG-13
1,Color,Gore Verbinski,Orlando Bloom,Johnny Depp,Pirates of the Caribbean: At World's End,Jack Davenport,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,English,USA,PG-13
2,Color,Sam Mendes,Rory Kinnear,Christoph Waltz,Spectre,Stephanie Sigman,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,English,UK,PG-13
3,Color,Christopher Nolan,Christian Bale,Tom Hardy,The Dark Knight Rises,Joseph Gordon-Levitt,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,English,USA,PG-13
5,Color,Andrew Stanton,Samantha Morton,Daryl Sabara,John Carter,Polly Walker,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,English,USA,PG-13
...,...,...,...,...,...,...,...,...,...,...
5026,Color,Olivier Assayas,Béatrice Dalle,Maggie Cheung,Clean,Don McKellar,http://www.imdb.com/title/tt0388838/?ref_=fn_t...,French,France,R
5027,Color,Jafar Panahi,Nargess Mamizadeh,Fereshteh Sadre Orafaiy,The Circle,Mojgan Faramarzi,http://www.imdb.com/title/tt0255094/?ref_=fn_t...,Persian,Iran,Not Rated
5033,Color,Shane Carruth,David Sullivan,Shane Carruth,Primer,Casey Gooden,http://www.imdb.com/title/tt0390384/?ref_=fn_t...,English,USA,PG-13
5035,Color,Robert Rodriguez,Peter Marquardt,Carlos Gallardo,El Mariachi,Consuelo Gómez,http://www.imdb.com/title/tt0104815/?ref_=fn_t...,Spanish,USA,R


### finding relevant columns to keep for analysis 

In [207]:
# Dropping string columns
movies_relevant_columns = movies_no_na.columns.drop(["facenumber_in_poster", "aspect_ratio", "movie_imdb_link","movie_color","actor_1_name","actor_2_name","director_name",
                                                    "movie_title","actor_3_name","movie_imdb_link","movie_language","country","content_rating"])
len(movies_relevant_columns)

8101

In [208]:
movies_clean = movies_no_na[movies_relevant_columns]

In [209]:
# Normalization
# x should not contain y
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import numpy as np
X = movies_clean
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns = X.columns)

/Users/lydialo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/lydialo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


### splitting the data into training and testing

In [215]:
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(42)

In [211]:
X = X.drop("imdb_score", axis = 1)
y = movies_clean.imdb_score

In [216]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

## Exploratory data analysis

In [ ]:
# Check for correlation between predictors using a confusion matrix 
# Helpful- https://jazpeng.github.io/predict_movie_ratings/

## One hot encoding

## Model generation

In [ ]:
# Logistic Regression
# Decision tree
# Lasso 
# Ridge 
# KNN classifier
# Random forest
# Compare to baseline accuracy score?
# Use GridSerach CV tool?

# action items

# questions

should we do more EDA to figure out how to clean this properly and / or to indicate what parts of the data we're losing when we drop NAs? (for example, if the NAs are mostly associated with older films or a particular genre of film)